Now that we have a SQLite database with indices, page titles, and coordinate strings, let's make a database where we extract all the metadata out of those coordinate strings so it's queryable.

This should be run after the other notebook that extracts the coordinate strings.

In [5]:
import csv
import json
from wikiparse import indexer, syntax_parser as sp
import time
import os
import sqlite3
import random

In [6]:
dumps = indexer.load_dumps(build_index=False, scratch_folder='py3')
english = dumps['en']

opening E:/enwiki-20190101-pages-articles-multistream.xml/scratch\py3\index.db
current mapping 19.1 m pages

__init__ complete


In [6]:
# english.db.close()

In [7]:
c = english.cursor

Before we create the database let's get a complete list of the entries we're going to want. That is, let's look at all the coordinate strings we've extracted from each page and extract the list of keywords from there. 

In [8]:
result = c.execute('''SELECT page_num,coords,title FROM indices WHERE coords != ""
''').fetchall()
coordStrings = {item[0]:item[1] for item in result}
idx_to_title = {item[0]:item[2] for item in result}
list(coordStrings.items())[:10]

[(66,
  'Coord|32.7|-86.7|type:adm1st_region:US_dim:1000000_source:USGS|display=title'),
 (86, 'Coord|36|42|N|3|13|E|type:city'),
 (114,
  'Coord|64|N|150|W|region:US-AK_type:adm1st_scale:10000000|display=title|notes=<ref>{{Cite gnis|1785533|State of Alaska'),
 (139,
  'Coord|13|19|N|169|9|W|type:event|name=Apollo 11 splashdown||Coord|10|36|N|172|24|E|display=inline||Coord|13|19|N|169|9|W|display=inline'),
 (140,
  'Coord|8|8|N|165|1|W|type:event|name=Apollo 8 landing||Coord|30|12|N|74|7|W|name=Apollo 8 S-IC impact||Coord|31|50|N|37|17|W|name=Apollo 8 S-II impact||Coord|8|8|N|165|1|W|name=Apollo 8 estimated splashdown'),
 (161,
  'Coord|12|30|40|N|69|58|27|W|type:isle|display=title||Coord|12|31|07|N|70|02|09|W||Coord|12|31|01|N|70|02|04|W|'),
 (168, 'Coord|12|30|S|18|30|E|display=title||Coord|8|50|S|13|20|E|type:city'),
 (177,
  'Coord|55|N|115|W|type:adm1st_scale:10000000_region:CA-AB|display=title'),
 (185, 'Coord|33|N|65|E|display=title||Coord|34|32|N|69|08|E|type:city'),
 (186,
  '

In [171]:
len(coordStrings), type(coordStrings)

(136417, dict)

In [172]:
list(idx_to_title_title.items())[:10]

[(66, 'Alabama'),
 (86, 'Algeria'),
 (114, 'Alaska'),
 (139, 'Apollo 11'),
 (140, 'Apollo 8'),
 (161, 'Aruba'),
 (168, 'Angola'),
 (177, 'Alberta'),
 (185, 'Afghanistan'),
 (186, 'Albania')]

If a page has more than one coordinate string, choose the one that's displayed at the top (`display=title`) or the first.

In [7]:
for page_num in coordStrings:
    if '||' in coordStrings[page_num]:
        pageCoordStrings = coordStrings[page_num].split('||')
        coordStrings[page_num] = pageCoordStrings[0]
        for s in pageCoordStrings:
            if "display=title" in s:
                coordStrings[page_num] = s

In [8]:
list(coordStrings.items())[:10]

[(66,
  'Coord|32.7|-86.7|type:adm1st_region:US_dim:1000000_source:USGS|display=title'),
 (86, 'Coord|36|42|N|3|13|E|type:city'),
 (114,
  'Coord|64|N|150|W|region:US-AK_type:adm1st_scale:10000000|display=title|notes=<ref>{{Cite gnis|1785533|State of Alaska'),
 (139, 'Coord|13|19|N|169|9|W|type:event|name=Apollo 11 splashdown'),
 (140, 'Coord|8|8|N|165|1|W|type:event|name=Apollo 8 landing'),
 (161, 'Coord|12|30|40|N|69|58|27|W|type:isle|display=title'),
 (168, 'Coord|12|30|S|18|30|E|display=title'),
 (177,
  'Coord|55|N|115|W|type:adm1st_scale:10000000_region:CA-AB|display=title'),
 (185, 'Coord|33|N|65|E|display=title'),
 (186, 'Coord|41|N|20|E|type:country_region:AL|display=title')]

For some Coord templates, there's a `note` (https://en.wikipedia.org/wiki/Template:Coord#Examples) which contains more pipes that will cut off the rest of the template. Since that tag seems to come after the other important tags, let's
ignore this problem.

In [9]:
def getKeywords(coordString, verbose=False):
    if verbose:
        print(coordString)
    keywords = {}
    rest = []
    items = coordString.split('|')
    for item in items:
        if '=' in item:
            keywords[item.split('=')[0]] = item.split('=')[1]
        elif ':' in item:
            keywords[item.split(':')[0]] = item.split(':')[1]
        else:
            rest.append(item)
#     return keywords, '|'.join(rest)
    return keywords
print(getKeywords('Coord|8|8|N|165|1|W|type:event|name=Apollo 8 landing'))
print(getKeywords('Coord|64|N|150|W|region:US-AK_type:adm1st_scale:10000000|display=title|notes=<ref>{{Cite gnis|1785533|State of Alaska'))
print(getKeywords('Coord|36|42|N|3|13|E|type:city'))

{'type': 'event', 'name': 'Apollo 8 landing'}
{'region': 'US-AK_type', 'display': 'title', 'notes': '<ref>{{Cite gnis'}
{'type': 'city'}


We only care about a subset of keywords, so let's make a whitelist.

In [10]:
from collections import defaultdict
keywords = defaultdict(int)
for coordString in coordStrings.values():
    for kw in getKeywords(coordString).keys():
        keywords[kw.strip()] += 1
keywords

defaultdict(int,
            {'type': 56689,
             'display': 119232,
             'region': 29135,
             'notes': 975,
             'name': 5800,
             'dim': 537,
             'globe': 228,
             'scale': 2092,
             'source': 4409,
             'format': 12041,
             'label': 1,
             'url': 68,
             'title': 76,
             'accessdate': 66,
             'DK_type': 1,
             "The ''globe''&nbsp;[[File": 25,
             'upright': 25,
             '': 1,
             'date': 29,
             'work': 42,
             'publisher': 56,
             'USGS': 1,
             'Register of Historic Parks and Gardens]].<ref name': 1,
             'num': 1,
             'desc': 1,
             'access-date': 25,
             'mode': 1,
             'deadurl': 2,
             'archiveurl': 1,
             'archivedate': 1,
             'df': 1,
             'landmark_region': 2,
             '<br>Crow Lane Roundabout': 1,
       

In [11]:
{kw for kw,count in keywords.items() if count > 10}

{"The ''globe''&nbsp;[[File",
 'access-date',
 'accessdate',
 'date',
 'dim',
 'display',
 'format',
 'globe',
 'id',
 'name',
 'notes',
 'publisher',
 'region',
 'scale',
 'source',
 'title',
 'type',
 'upright',
 'url',
 'work'}

Most of these look good except for a few that are duplicates or a weird artifact from the imperfect string processing. 

In [12]:
entriesList = [
 'accessdate',
 'date',
 'dim',
 'display',
 'elevation',
 'format',
 'globe',
 'id',
 'name',
 'nosave',
 'notes',
 'publisher',
 'reason',
 'region',
 'scale',
 'source',
 'title',
 'type',
 'upright',
 'url',
 'work']
for e in entriesList:
    print(e, 'TEXT,')

accessdate TEXT,
date TEXT,
dim TEXT,
display TEXT,
elevation TEXT,
format TEXT,
globe TEXT,
id TEXT,
name TEXT,
nosave TEXT,
notes TEXT,
publisher TEXT,
reason TEXT,
region TEXT,
scale TEXT,
source TEXT,
title TEXT,
type TEXT,
upright TEXT,
url TEXT,
work TEXT,


It would be nice to see examples of values for the keywords so we can see how they're used.

In [13]:
from numpy.random import shuffle
def findKeywordExample(kw, count=1, debug=False):
    kws = []
    coordStringsList = list(coordStrings.values())
    shuffle(coordStringsList)
    for cs in coordStringsList:
        if f'|{kw}=' in cs:
            kws.append(cs.split(f'|{kw}=')[1].split('|')[0])
            if debug:
                print(cs)
        elif f'|{kw}:' in cs:
            kws.append(cs.split(f'|{kw}:')[1].split('|')[0])
            if debug:
                print(cs)
        if len(kws) >= count:
            break
    return kws

In [14]:
findKeywordExample('name', 10, True)

Coord|49|36|N|2|53|W|name=Britannic
Coord|43|02|11|N|83|26|24|W|region:US-MI_type:city_scale:30000_source:GNIS|name=Elba
Coord|54|21|15|N|6|01|30|E|type:landmark|name=BorWin1 - Borkum 2 Platform
Coord|42.74196953|-90.32643556|type:edu_scale:3000_region:US|display=title|name=Belmont High School (Belmont, Wisconsin)
Coord|42|27|3.378|N|84|11|11.804|W|dim:250_region:US-MI_type:edu|name=Stockbridge Community Schools Central Office|display=inline,title
Coord|50|56|43|N|005|03|01|E|name=Blakenberg Airfield
Coord|45|05|30|N|069|05|42|W|name=Charleston AFS P-65|display=inline,title
Coord|39|41|51|N|96|40|46|W|format=dms|display=title|name=Little Blue River|type:river_region:US-KS
Coord|51.1384|N|2.7358|W|display=inline|region:GB-SOM_type:landmark|format=dms|name=Start point at River Brue
Coord|38|19|13|N|123|2|6|W|type:landmark_region:US-CA|name=Bird Walk


['Britannic',
 'Elba',
 'BorWin1 - Borkum 2 Platform',
 'Belmont High School (Belmont, Wisconsin)',
 'Stockbridge Community Schools Central Office',
 'Blakenberg Airfield',
 'Charleston AFS P-65',
 'Little Blue River',
 'Start point at River Brue',
 'Bird Walk']

Now we can create all the columns en masse.

In [174]:
c.execute('''CREATE TABLE coords
    (coords TEXT,
    lat REAL DEFAULT 0,
    lon REAL DEFAULT 0,
    page_num INTEGER PRIMARY KEY,
    accessdate TEXT DEFAULT '',
    date TEXT DEFAULT '',
    dim TEXT DEFAULT '',
    display TEXT DEFAULT '',
    elevation TEXT DEFAULT '',
    format TEXT DEFAULT '',
    globe TEXT DEFAULT '',
    id TEXT DEFAULT '',
    name TEXT DEFAULT '',
    nosave TEXT DEFAULT '',
    notes TEXT DEFAULT '',
    publisher TEXT DEFAULT '',
    reason TEXT DEFAULT '',
    region TEXT DEFAULT '',
    scale TEXT DEFAULT '',
    source TEXT DEFAULT '',
    title TEXT DEFAULT '',
    type TEXT DEFAULT '',
    upright TEXT DEFAULT '',
    url TEXT DEFAULT '',
    work TEXT DEFAULT '')
''')

In [173]:
# c.execute('''DROP TABLE coords''')

In [39]:
def extract_lat_lon(coord_string):
    split = coord_string.split('|')
    coord_list = []
    for s in split:
        if ':' in s or '=' in s:
            break
        if 'Coord' not in s and 'LAT' not in s and 'LONG' not in s:
            coord_list.append(s)  
    return coord_list

begin = random.randint(0, len(coordStrings)-10)
for i in range(begin, begin+10):
    print(extract_lat_lon(list(coordStrings.values())[i]))

['44', '38', '9.44', 'N', '63', '35', '25.17', 'W']
['40', '42', '31', 'N', '74', '00', '36', 'W']
['23.7186', 'N', '90.3981', 'E']
['44.8951', '-93.2608']
['44.298775', '-120.141648']
['22.7277', '114.2661']
[]
['45', '20', '48.97', 'N', '75', '45', '33.79', 'W']
['45', '25', '39.1', 'N', '75', '41', '28.2', 'W']
['45', '25', '39.1', 'N', '75', '41', '28.2', 'W']


In [146]:
def convert_to_decimal(coord_list):
#     print(' '.join(coord_list), end='\t')
    coord_list = [s.strip().lower() for s in coord_list if s.strip() != '']
    if len(coord_list) < 2:
        return [0, 0]
    if len(coord_list) == 2:
        return [float(coord_list[0]), float(coord_list[1])]
    directions = 0
    for s in coord_list:
        s = s.strip().lower()
        if s and s.strip() in 'nesw':
            directions += 1
    if directions != 2:
        raise Exception(directions, "wrong number of directions for:", coord_list)
        
    lat = []
    lon = []
    creating_lat = True
    for s in coord_list:
        s = s.strip().lower()
        if s == '':
            continue
        if creating_lat:
            if s in 'ns':
                creating_lat = False
                while len(lat) < 3:
                    lat.append(0)
                if s == 'n':
                    lat.append(1)
                else:
                    lat.append(-1)
            else:
                if ',' in s:
                    s = s.replace(',', '.')
                lat.append(float(s))
        else:
            if s in 'ew':
                while len(lon) < 3:
                    lon.append(0)
                if s == 'e':
                    lon.append(1)
                else:
                    lon.append(-1)
            else:
                if ',' in s:
                    s = s.replace(',', '.')
                lon.append(float(s))
    return [
        (lat[0] + lat[1]/60 + lat[2]/3600) * lat[3],
        (lon[0] + lon[1]/60 + lon[2]/3600) * lon[3]
    ]

count = 1000
begin = random.randint(0, len(coordStrings)-count)
for i in range(begin, begin+count):
    coord_list = extract_lat_lon(list(coordStrings.values())[i])
    try:
        result = convert_to_decimal(coord_list)
    except Exception as e:
        print(coord_list, e)
#     print(result)

['{{Infobox UK postcode area/stats', '{{{area_code', 'WC'] (0, 'wrong number of directions for:', ['{{infobox uk postcode area/stats', '{{{area_code', 'wc'])
["''lat''", "''long''"] could not convert string to float: "''lat''"
['latitude', 'longitude'] could not convert string to float: 'latitude'
['latitude', 'longitude'] could not convert string to float: 'latitude'
['latitude', 'longitude'] could not convert string to float: 'latitude'
['coord]]', '1', '2'] (0, 'wrong number of directions for:', ['coord]]', '1', '2'])
['latitude', 'longitude'] could not convert string to float: 'latitude'
['latitude', 'longitude'] could not convert string to float: 'latitude'
['latitude', 'longitude'] could not convert string to float: 'latitude'


In [153]:
def insertKeywordDict(page_num, coords, kws, debug=False):
    allowed_keys = [kw for kw in kws.keys() if kw in entriesList]
    allowed_vals = [kws[kw] for kw in allowed_keys]
    
    try:
        lat,lon = convert_to_decimal(extract_lat_lon(coords))
    except:
        lat = lon = 0
    keys = ['coords', 'lat', 'lon', 'page_num'] + allowed_keys
    vals = [coords, lat, lon, page_num] + allowed_vals
    
    qstring = f'INSERT INTO coords (\
        {",".join(keys)}) VALUES (\
        {",".join(["?" for item in vals])})'
    
    if debug:
        print(qstring, vals)
    c.execute(qstring, vals)
# insertKeywordDict({'region': 'US-AK_type', 'display': 'title', 'notes': '<ref>{{Cite gnis'})

Grab a drink before running the next cell, it'll take a while.

In [175]:
count = 0
for page_idx,coordString in coordStrings.items():
    keywords = getKeywords(coordString)
    keywords['title'] = idx_to_title[page_idx]
    insertKeywordDict(page_idx, coordString, keywords)
    if count > 100:
        english.db.commit()
        print(round(100*count/len(coordStrings), 2), end='\r')
    count += 1

In [178]:
# c.execute('''SELECT * FROM coords WHERE page_num > 19000000''').fetchall()

In [23]:
l = c.execute('''SELECT title FROM coords WHERE 
        
         lat BETWEEN 47.6 AND 47.7
        AND lon BETWEEN -122.35 AND -122.34
    ''').fetchall()
l

[('Space Needle',),
 ('Woodland Park Zoo',),
 ('Belltown, Seattle',),
 ('Cranium, Inc.',),
 ('Seattle Aquarium',),
 ('Attachmate',),
 ('Unexpected Productions',),
 ('Memorial Stadium (Seattle)',),
 ('Cutter & Buck',),
 ('Mercer Arena',),
 ('Seattle Cinerama',),
 ("Beth's Cafe",),
 ('Pike Place Fish Market',),
 ('Flight to Mars (ride)',),
 ('InterConnection.org',),
 ('A Place for Mom',),
 ('Eastern Congo Initiative',),
 ('Institute for Health Metrics and Evaluation',),
 ('Impinj',),
 ('Griffin College',),
 ('Lundeberg Derby Monument',),
 ('Canlis',),
 ('Original Starbucks',),
 ('Pier 57 (Seattle)',),
 ('Fremont Brewing',),
 ('Bell Street Park',),
 ('Arrivé',),
 ('Tower 12',),
 ('Left Bank Books',)]

In [27]:
english.cursor.execute('''SELECT * FROM titles WHERE 
title IN (SELECT title FROM coords WHERE 
          lat BETWEEN 47.6 AND 47.7
          AND lon BETWEEN -122.35 AND -122.34)
''').fetchall()

[]

In [30]:
english.cursor.execute('''DROP TABLE titles
''').fetchall()

[]